In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import CustomerPrepreocess

In [2]:
df_cus_all = pd.read_csv('data/Customer.csv'
,usecols=['CustomerCode','IsOrder','IsDeal','OrderAgentID']
,dtype={'OrderAgentID':str})
df_sale = pd.read_excel('data/sales.xlsx',dtype={'AgentID':str})

In [3]:
df_cus = pd.read_csv('data/Customer.csv',usecols=
['CustomerCode','IsOrder','ProjectRouteCover'
 ,'KIStatus','Gender','StudyTarget2','JobType'
 ,'WatchTimes','LearningPurpose','ActionForce'
 ,'CustomerExpRating','Age']
,dtype={"LearningPurpose": str,'ActionForce':str})
df_cus_trim=CustomerPrepreocess.process(df_cus,'train')

Using TensorFlow backend.


In [4]:
df_cus_all=df_cus_all[df_cus_all.CustomerCode.isin(df_cus_trim.CustomerCode)==True]

In [5]:
df_cus=df_cus_all

In [6]:
df_cus.shape

(216958, 4)

# data

In [7]:
def order_customer_feature(df_cus,customer_list):
    df_cus=df_cus.set_index('CustomerCode')
    df_cus=df_cus.reindex(customer_list)
    df_cus=df_cus.reset_index()
    return df_cus
def align_ID(id):
    return id.rjust(6,'0')
df_sale.AgentID=df_sale.AgentID.apply(align_ID)
customer_list=df_cus.CustomerCode.unique().tolist()
sale_list=df_sale.AgentID.unique().tolist()

In [8]:
df_cus.IsOrder=df_cus.IsOrder.fillna(0)
df_cus.IsDeal=df_cus.IsDeal.fillna(0)

In [9]:
row_index  = []
col_index  = []
data = []
list_data=df_cus.values.tolist()
col=list(df_cus.columns.values)
list_len=len(list_data)
for idx, val in enumerate(list_data):
#     print(list_data[idx])
    row=dict(zip(col,list_data[idx]))
    if (((row['IsOrder']>0) or (row['IsDeal']>0)) and (row['OrderAgentID'] in sale_list)):
        sale_index=sale_list.index(row['OrderAgentID'])
        row_index.append(sale_index)
        col_index.append(idx)
        data.append(1)

In [10]:
sparse_data=coo_matrix((data, (row_index, col_index)))

In [11]:
sparse_data.toarray().shape

(454, 216935)

# order feature(customer)

In [12]:
def order_customer_feature(df_cus,customer_list):
    df_cus=df_cus.set_index('CustomerCode')
    df_cus=df_cus.reindex(customer_list)
    df_cus=df_cus.reset_index()
    return df_cus

In [13]:
customer_feature=order_customer_feature(df_cus_trim,customer_list)

In [14]:
customer_feature=customer_feature.set_index('CustomerCode')

In [15]:
row_index  = []
col_index  = []
data = []
list_data=customer_feature.values.tolist()
col=list(customer_feature.columns.values)
list_len=len(list_data)
for idx, val in enumerate(list_data):
    for idx_2, val_2 in enumerate(val):
        if(val_2>0):
            row_index.append(idx)
            col_index.append(idx_2)
            data.append(val_2)

In [16]:
sparse_customer_feature=coo_matrix((data, (row_index, col_index))).tocsr()

In [17]:
type(sparse_customer_feature)

scipy.sparse.csr.csr_matrix

In [18]:
sparse_customer_feature.toarray().shape

(216958, 55)

# saler feature

# train

In [19]:
from lightfm import cross_validation
train,test=cross_validation.random_train_test_split(sparse_data,0.1)

In [20]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(loss='warp-kos')
model.fit(sparse_data, epochs=10, num_threads=4)

# test

In [21]:
print('train auc: ',auc_score(model,train,num_threads=2).mean())
print('test auc:',auc_score(model,test, train_interactions=train, num_threads=2).mean())

train auc:  0.9145238
test auc: 0.8542552


In [22]:
precision_at_k(model, test, k=10).mean()

0.061077844

In [23]:
precision_at_k(model, train, k=10).mean()

0.34082398